In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools
from numba import njit
from joblib import Parallel, delayed
from Utils import extract_symbol_timeframe

# === Numba ускорение ===
@njit
def ema(arr, window):
    alpha = 2 / (window + 1)
    result = np.empty_like(arr)
    result[0] = arr[0]
    for i in range(1, len(arr)):
        result[i] = alpha * arr[i] + (1 - alpha) * result[i - 1]
    return result

@njit
def macd_calc(close, fast, slow, signal):
    ema_fast = ema(close, fast)
    ema_slow = ema(close, slow)
    macd = ema_fast - ema_slow
    macd_signal = ema(macd, signal)
    return macd, macd_signal

# === Основная логика обработки одной комбинации ===
def process_combination(fast, slow, signal, tp, close_all, high_all, low_all, time_all):
    if fast >= slow:
        return None

    macd, macd_signal = macd_calc(close_all, fast, slow, signal)

    # ✅ Пересечение снизу вверх + MACD ниже нуля
    cross_up = (macd[:-1] < macd_signal[:-1]) & (macd[1:] > macd_signal[1:]) & (macd[1:] < 0)
    entry_idx = np.where(cross_up)[0] + 1

    balance = 10.0
    durations = []
    entry_times = []
    num_trades = 0
    last_exit = 0
    stop_triggered = False
    first_stop_time = pd.NaT
    stop_loss_pct = 99

    for i in entry_idx:
        if i <= last_exit or i >= len(close_all):
            continue

        entry_price = close_all[i]
        take_profit = entry_price * (1 + tp / 100)
        stop_loss = entry_price * (1 - stop_loss_pct / 100)
        entry_time = time_all[i]

        for j in range(i + 1, len(close_all)):
            if high_all[j] >= take_profit:
                exit_price = take_profit
                exit_time = time_all[j]
                break
            elif low_all[j] <= stop_loss:
                exit_price = stop_loss
                exit_time = time_all[j]
                if not stop_triggered:
                    stop_triggered = True
                    first_stop_time = exit_time
                break
        else:
            break

        profit_pct = (exit_price / entry_price - 1) * 100
        balance *= (1 + profit_pct / 100)
        duration = (exit_time - entry_time).astype("timedelta64[m]").astype(int)

        durations.append(duration)
        entry_times.append(entry_time)
        num_trades += 1
        last_exit = j

    if num_trades == 0:
        return None

    return {
        "fast_range": fast,
        "slow_range": slow,
        "signal_range": signal,
        "tp_range": tp,
        "profit_total_pct": round((balance / 10 - 1) * 100, 2),
        "num_trades": num_trades,
        "min_time": int(np.min(durations)),
        "max_time": int(np.max(durations)),
        "mean_time": round(np.mean(durations), 2),
        "median_time": int(np.median(durations)),
        "last_entry_time": entry_times[-1],
        "stop_triggered": stop_triggered,
        "stop_time": first_stop_time if stop_triggered else pd.NaT
    }

# === Загрузка данных ===

path = " " # Format filename = {symbol}USDT_timeframe_data.csv, example: "BTCUSDT_1h_from_1_Jan_2020.csv"
df = pd.read_csv(path)

df["open_time"] = pd.to_datetime(df["open_time"], utc=True)
close_all = df["close"].to_numpy(dtype=np.float32)
high_all = df["high"].to_numpy(dtype=np.float32)
low_all = df["low"].to_numpy(dtype=np.float32)
time_all = df["open_time"].to_numpy(dtype="datetime64[m]")

# === Параметры ===

symbol, timeframe = extract_symbol_timeframe(path)
indicator_name = "MACD_below_0"

fast_range = range(6, 22)
slow_range = range(20, 41)
signal_range = range(5, 16)

if timeframe in ["1m", "5m", "15m"]:
    tp_range = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.5, 1.7, 2, 2.5, 3] #for timeframe 1m - 5m - 15m - 1h
else:
    tp_range = tp_range = [0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.5, 2, 2.5, 3] + list(range(4, 11)) #for timeframe 1h -4h

combos = list(itertools.product(fast_range, slow_range, signal_range, tp_range))

# === Параллельная обработка ===
results = Parallel(n_jobs=-1)(
    delayed(process_combination)(fast, slow, signal, tp, close_all, high_all, low_all, time_all)
    for fast, slow, signal, tp in tqdm(combos, desc="⚙️ Grid Search MACD")
)

# === Сохранение результатов ===
results = [r for r in results if r is not None]
result_df = pd.DataFrame(results).sort_values("profit_total_pct", ascending=False)

out_name = f"{symbol}_{timeframe}_{indicator_name}_gridsearch_{len(combos)}.csv"
result_df.to_csv(out_name, index=False)
print(f"📁 Результаты сохранены в: {out_name}")
print(f"🏆 Топ-5 лучших стратегий для {symbol} на таймфрейме {timeframe}:")
result_df.head(5)


⚙️ Grid Search MACD:   6%|▌         | 3480/62832 [00:30<03:20, 295.87it/s]